In [ ]:
%pip install transformer_lens
%pip install wandb

def in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

IN_COLAB = in_colab()

if IN_COLAB:
    # download files from the github repository
    for file in ['model.py', 'replay.py', 'trainer.py', 'utils.py']:
      !wget https://raw.githubusercontent.com/jacksonkunde/rlhf/main/{file}

In [4]:
from trainer import RLHFTrainer
from utils import RLHFTrainingArgs

import torch as t
import torch.nn as nn
from torch import Tensor
import wandb
from transformer_lens.hook_points import HookPoint
from transformer_lens import utils, HookedTransformer
from typing import List, Optional, Tuple, Union, Dict, Any, Callable
import einops
from jaxtyping import Float, Int
import os
import sys
from pathlib import Path
from rich import print as rprint
from rich.table import Table
from dataclasses import dataclass
import numpy as np
import time
from functools import partial

In [5]:
def reward_fn_char_count(generated_sample: Union[str, List[str]], char: str = '.') -> Union[float, Float[Tensor, "batch"]]:
    '''
    Reward function, evaluated on the generated samples.

    In this case it's very simple: it just counts the number of instances of a particular character in
    the generated sample. It returns a tensor of rewards of dtype float the input is a list, or a single
    reward (float) if the input is a string.
    '''
    if isinstance(generated_sample, list):
        return t.tensor([reward_fn_char_count(item) for item in generated_sample]).float().to("cuda")
    else:
        return float(generated_sample.count(char))

In [ ]:
args = RLHFTrainingArgs(use_wandb=False, reward_fn=reward_fn_char_count)
trainer = RLHFTrainer(args)
trainer.train()